In [2]:
import os
import copy
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import random_split, DataLoader
from sklearn.metrics import classification_report

In [3]:
# Configs
data_dir = 'dataset'
batch_size = 32
num_epochs = 10
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_save_path = 'best_model.pth'
num_classes = 3  # dry, normal, oily

In [4]:
# Transforms
mean = [0.485, 0.456, 0.406]  # ImageNet mean
std = [0.229, 0.224, 0.225]   # ImageNet std

In [5]:
# With augmentation
data_transforms_aug = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
}

In [6]:
# Without augmentation
data_transforms_plain = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
}

In [7]:
def load_datasets(transforms_dict):
    full_dataset = datasets.ImageFolder(data_dir, transform=transforms_dict['train'])
    val_dataset = datasets.ImageFolder(data_dir, transform=transforms_dict['val'])

    train_size = int(0.8 * len(full_dataset))
    val_size = len(full_dataset) - train_size

    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader

In [9]:
def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs, mode_name):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'\n[{mode_name}] Epoch {epoch + 1}/{num_epochs}')
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Save best model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), model_save_path)
                print("✅ Saved Best Model")

        scheduler.step()
        
    print(f'\nBest Validation Acc: {best_acc:.4f}')
    model.load_state_dict(best_model_wts)
    return model

In [10]:
def test_model(model, dataloader):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=['dry', 'normal', 'oily']))

In [11]:
# Function to run entire training
def run_pipeline(use_aug=True):
    print("\n" + "=" * 60)
    print("Using Augmentation" if use_aug else "Without Augmentation")
    print("=" * 60)

    transforms_dict = data_transforms_aug if use_aug else data_transforms_plain
    train_loader, val_loader = load_datasets(transforms_dict)
    dataloaders = {'train': train_loader, 'val': val_loader}

    # Load pretrained ResNet18
    model_ft = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, num_classes)
    model_ft = model_ft.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model_ft.parameters(), lr=learning_rate)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

    model_ft = train_model(model_ft, dataloaders, criterion, optimizer, scheduler, num_epochs,
                           mode_name="Aug" if use_aug else "Plain")

    # Test model
    print("\nTesting on Test Set")
    _, test_loader = load_datasets(transforms_dict)
    test_model(model_ft, test_loader)

In [12]:
run_pipeline(use_aug=False)


Without Augmentation


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\hp/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100.0%



[Plain] Epoch 1/10


d:\sadia\resnet\model\venv\Lib\site-packages\PIL\Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Train Loss: 0.6722 Acc: 0.7265
Val Loss: 0.7975 Acc: 0.7377
✅ Saved Best Model

[Plain] Epoch 2/10
Train Loss: 0.5022 Acc: 0.7793
Val Loss: 0.5972 Acc: 0.7761
✅ Saved Best Model

[Plain] Epoch 3/10
Train Loss: 0.4554 Acc: 0.8161
Val Loss: 0.7588 Acc: 0.7143

[Plain] Epoch 4/10
Train Loss: 0.4440 Acc: 0.8198
Val Loss: 0.6519 Acc: 0.7399

[Plain] Epoch 5/10
Train Loss: 0.4168 Acc: 0.8342
Val Loss: 0.5673 Acc: 0.7719

[Plain] Epoch 6/10
Train Loss: 0.2870 Acc: 0.8897
Val Loss: 0.4401 Acc: 0.8465
✅ Saved Best Model

[Plain] Epoch 7/10
Train Loss: 0.2206 Acc: 0.9174
Val Loss: 0.4347 Acc: 0.8401

[Plain] Epoch 8/10
Train Loss: 0.1772 Acc: 0.9366
Val Loss: 0.4615 Acc: 0.8443

[Plain] Epoch 9/10
Train Loss: 0.1420 Acc: 0.9462
Val Loss: 0.4609 Acc: 0.8614
✅ Saved Best Model

[Plain] Epoch 10/10
Train Loss: 0.1237 Acc: 0.9590
Val Loss: 0.4735 Acc: 0.8550

Best Validation Acc: 0.8614

Testing on Test Set

Classification Report:
              precision    recall  f1-score   support

         dry  